# Setup

In [1]:
!pip -q install imagecodecs 

     |████████████████████████████████| 30.0MB 93kB/s 


In [4]:
!pip install catboost=='0.26'

     |████████████████████████████████| 69.2MB 41kB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!cp "/content/drive/MyDrive/lacuna_zindi/lacuna/train_unique.csv" .
!cp "/content/drive/MyDrive/lacuna_zindi/lacuna/auxiliary_data_unique.csv" . 
!cp "/content/drive/MyDrive/lacuna_zindi/lacuna/extra_train.csv" . 
!cp "/content/drive/MyDrive/lacuna_zindi/lacuna/test.csv" .


In [8]:

!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/sentinel-part1.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/sentinel_part2.zip"

!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/extra_train_sentinel.zip"

!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/sentinel_for_points_collected_in_2015.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/sentinel_for_points_collected_in_2015_updated_2.zip"
#ATTENTION: when the message replace sentinel_for_points_collected_in_2015 shows, press A 

replace sentinel_for_points_collected_in_2015/cdc19ade.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [9]:
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/planet_june18.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/planet_june17.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/planet_dec18.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/planet_dec17.zip"

!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/extra_train_planet_jun18.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/extra_train_planet_dec18.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/extra_train_planet_jun17.zip"
!unzip -q "/content/drive/MyDrive/lacuna_zindi/lacuna/extra_train_planet_dec17.zip"

In [10]:
import os

import pandas as pd
import numpy as np

import skimage.io
import matplotlib.pylab as plt
import scipy

from scipy import ndimage
from skimage.segmentation import quickshift

# Loading data

In [11]:
#function for loading RGB image from planet lab sattelite
def load_RGB_images(ID, load_extra=False):
  if load_extra:
    extra = 'extra_train-'
  else:
    extra = ''
  name = ID.split('_')[1]
  img_jun17 = skimage.io.imread(f'{extra}planet-jun17/{name}.png')
  img_dec17 = skimage.io.imread(f'{extra}planet-dec17/{name}.png')
  img_jun18 = skimage.io.imread(f'{extra}planet-jun18/{name}.png')
  img_dec18 = skimage.io.imread(f'{extra}planet-dec18/{name}.png')
  return img_jun17, img_dec17, img_jun18, img_dec18

In [12]:
#function for loading spectral bands from sentinel 2 sattelite
def load_Spectral_image(ID, load_extra=False, year_2015=False):
  # e.g id_0b242e06 -> 0b242e06
  if load_extra:
    extra = 'extra_train-'
  else:
    extra = ''
  name = ID.split('_')[1]
  if year_2015:
    root_dir = 'sentinel_for_points_collected_in_2015'
  else:
    root_dir = extra + 'sentinel'
  img_sentinel = skimage.io.imread(f'{root_dir}/{name}.tif')
  return img_sentinel

In [13]:

#load all data
train = pd.read_csv('train_unique.csv')
aux = pd.read_csv('auxiliary_data_unique.csv')
extra = pd.read_csv('extra_train.csv')
test = pd.read_csv('test.csv')

#concatenate train, aux and extra 
TT=pd.concat([train,aux,extra])

# Feature engineering

In [14]:
#sliding_window returns a list of 16 (3*3) windows around the center of the image
def sliding_window(im,x0,y0):
  set1=im[x0-2:x0+1,y0-2:y0+1]
  set2=im[x0-5:x0-2,y0-2:y0+1]
  set3=im[x0-2:x0+1,y0-5:y0-2]
  set4=im[x0-5:x0-2,y0-5:y0-2]
  set5=im[x0:x0+3,y0-2:y0+1]
  set6=im[x0+3:x0+6,y0-2:y0+1]
  set7=im[x0:x0+3,y0-5:y0-2]
  set8=im[x0+3:x0+6,y0-5:y0-2]
  set9=im[x0:x0+3,y0:y0+3]
  set10=im[x0:x0+3,y0+3:y0+6]
  set11=im[x0+3:x0+6,y0:y0+3]
  set12=im[x0+3:x0+6,y0+3:y0+6]
  set13=im[x0-2:x0+1,y0:y0+3]
  set14=im[x0-5:x0-2,y0+3:y0+6]
  set15=im[x0-2:x0+1,y0+3:y0+6]
  set16=im[x0-5:x0-2,y0:y0+3]

  sets=[set1,set2,set3,set4,set5,set6,set7,set8,set9,set10,set11,set12,set13,set14,set15,set16]

  return sets

In [15]:
def im_stats(spec_im,planet):
  
  statistics={}
  
  x0, y0 = spec_im.shape[1]//2, spec_im.shape[0]//2
  
  redband_stack=np.zeros(spec_im.shape)
  NIRband_stack=np.zeros(spec_im.shape)
  ndvi_im_stack=np.zeros(spec_im.shape)
  
  
  for month in range(12):

    redband_stack[:,:,month]=spec_im[:,:,3+16*month].astype('float64')
    NIRband_stack[:,:,month]=spec_im[:,:,7+16*month].astype('float64')
    ndvi_im_stack[:,:,month]=(NIRband_stack[:,:,month]-redband_stack[:,:,month])/(redband_stack[:,:,month]+NIRband_stack[:,:,month])
   
    
    ndvi_im=ndvi_im_stack[:,:,month]

 # statistics on ndvi for 3*3 rolling windows 
    windows=sliding_window(ndvi_im,x0,y0)
    
    for i in range(len(windows)):
      
      statistics[f'{i}_local_ndvi_{month}']=np.median(windows[i].flatten()) #ndvi_median in each window
      statistics[f'{i}_local_range_{month}']=np.max(windows[i].flatten())-np.min(windows[i].flatten()) #ndvi range between max and min ndvi values in a window

   
    for i in range(len(windows)):
      
      statistics[f'{i}_nbg_{month}']=len(windows[i][windows[i]>=0.6].flatten())#ndvi >=0.6 indicates the presence of vegetation
      statistics[f'{i}_nbs_{month}']=len(windows[i][windows[i]<=0.1].flatten()) #ndvi <=0.1 indicates the presence for bare soil/built up areas
      
      statistics[f'{i}_ratio_{month}']=ndvi_im[x0,y0]/statistics[f'{i}_local_ndvi_{month}']

       
#segmentation : using the quickshift algorithm to delineate(approximately) fields boundaries 
#and calculate the center of mass of the field in wich the center is located


  # Calculate the segments using the quickshift algorithm with kernel_size 3 and 4 for each planet lab image
   
  for size in [3,4]:
    mean_x=0
    mean_y=0
 
    for i in range(len(planet)):
      
      x0_rgb, y0_rgb=planet[i].shape[1]//2, planet[i].shape[0]//2
      segments = quickshift(planet[i],
                          kernel_size=size,
                          convert2lab=False,
                          max_dist=5,
                          ratio=1)

      
      mask=np.where(segments==segments[x0_rgb,y0_rgb],1,0)
      

      statistics[f'cluster_center_x_{i}_size{size}']=ndimage.measurements.center_of_mass(mask)[0]
      mean_x=mean_x+statistics[f'cluster_center_x_{i}_size{size}']
      statistics[f'cluster_center_y_{i}_size{size}']=ndimage.measurements.center_of_mass(mask)[1] 
      mean_y=mean_y+statistics[f'cluster_center_y_{i}_size{size}']

    
    mean_x=mean_x/len(planet)
    mean_y=mean_y/len(planet)

    statistics[f'mean_x_size{size}']=mean_x
    statistics[f'mean_y_size{size}']=mean_y
  

# calculating temporal std in ndvi_im : ndvi_im is calculated in each of the 12 months 
# then for each pixel std ndvi from data of the 12 months is calculated 
# --> pixels with high std indicates changing areas over time therefore pixels with low std are most probably roads or homes  
  std_im=np.std(ndvi_im_stack,axis=2)
  sets=sliding_window(std_im,x0,y0) # statistics:[median,max] from std_im are calculated for the 16 windows (3*3)
  
  for i in range(len(sets)):
    statistics[f'{i}_std_window']=np.median(sets[i].flatten())
    statistics[f'{i}_max_window']=np.max(sets[i].flatten())  
    
  
  return statistics

**creating train_NDVI**

the quality column is dropped because it is not available in test

In [16]:
import tqdm 
from tqdm import tqdm_notebook as tqdm 
train_NDVI = pd.DataFrame([im_stats(load_Spectral_image(TT.ID.values[fid_idx],load_extra=(fid_idx>=1022), year_2015=(TT.Year.values[fid_idx]==2015)),
                                     load_RGB_images(TT.ID.values[fid_idx],load_extra=(fid_idx>=1022))) for fid_idx in tqdm(range(len(TT['ID'].values))) ])
train_NDVI['ID'] = TT['ID'].values

train_NDVI=pd.merge(train_NDVI,TT,on='ID',how='left')

train_NDVI.drop('Quality',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in double_scalars


**creating test_NDVI**

In [17]:
test_NDVI = pd.DataFrame([im_stats(load_Spectral_image(test.ID.values[fid_idx], year_2015=(test.Year.values[fid_idx]==2015)),
                                     load_RGB_images(test.ID.values[fid_idx])) for fid_idx in tqdm(range(len(test['ID'].values))) ])
test_NDVI['ID'] = test['ID'].values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in double_scalars


Yield is dropped because it contains many nans 

np.inf caused by devision by zero are imputed with 0 

In [18]:
test_NDVI=pd.merge(test_NDVI,test,on='ID',how='left')

test_NDVI.drop('Yield',axis=1,inplace=True)
train_NDVI.drop('Yield',axis=1,inplace=True)

#correcting the divide by zero warning when creating train_NDVI and test_NDVI with imputing np.inf with zero
train_NDVI.replace(np.inf,0,inplace=True)
test_NDVI.replace(np.inf,0,inplace=True)

# training (lgbm)

In [25]:
import lightgbm as lgb 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

skf = KFold(n_splits=3)
X=train_NDVI.drop(['x','y','ID'],axis=1)
y1=train_NDVI.x
y2=train_NDVI.y
ntest=test_NDVI.drop('ID',axis=1)
list_preds_lg1=[]
list_preds_lg2=[]


oof_preds_lg = np.zeros((train_NDVI.shape[0],2))

for nb ,  (train_index, test_index) in enumerate(skf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y1_train, y1_test = y1.iloc[train_index], y1.iloc[test_index]
    y2_train, y2_test = y2.iloc[train_index], y2.iloc[test_index]
    
    
    model1_lg = lgb.LGBMRegressor(n_estimators=10000,learning_rate=0.01,max_depth=10,eval_metric='mae',metric= 'mae',random_seed=0,feature_fraction=0.7)
    model1_lg.fit(X_train,y1_train,eval_set = [(X_test, y1_test)], early_stopping_rounds  = 300,verbose=0) 
    
    preds1_lg=model1_lg.predict(X_test)
    oof_preds_lg[test_index,0] = preds1_lg
    

    model2_lg=lgb.LGBMRegressor(n_estimators=10000,learning_rate=0.01,max_depth=10,eval_metric='mae',metric= 'mae',random_seed=0,feature_fraction=0.7)
    model2_lg.fit(X_train,y2_train,eval_set=[(X_test,y2_test)],verbose = False,early_stopping_rounds=300)
    
    preds2_lg=model2_lg.predict(X_test)
    oof_preds_lg[test_index,1] = preds2_lg
    
    tmp=pd.DataFrame({'y1':y1_test,'y2':y2_test,'preds1':preds1_lg,'preds2':preds2_lg})
    print("Fold [{}] : {}".format(nb,mean_absolute_error(tmp[['y1','y2']], tmp[['preds1','preds2']])))
    
    list_preds_lg1.append(model1_lg.predict(ntest))
    list_preds_lg2.append(model2_lg.predict(ntest))

Fold [0] : 0.19348574365262822
Fold [1] : 0.2036912778522417
Fold [2] : 0.2315218282451731


In [26]:
tmp_lg=train_NDVI[['ID','x','y']].copy()
tmp_lg['preds_x']=oof_preds_lg[:,0]
tmp_lg['preds_y']=oof_preds_lg[:,1]

mean_absolute_error(tmp_lg[['x','y']], tmp_lg[['preds_x','preds_y']])

0.20955541954640466

In [27]:
m2_lgbm_test=test_NDVI[['ID']].copy()
m2_lgbm_test['x']=np.mean(list_preds_lg1,axis=0)
m2_lgbm_test['y']=np.mean(list_preds_lg2,axis=0)

In [28]:
m2_lgbm_test.head(5)

,ID,x,y
0,id_e7032b10,-0.024195,0.002737
1,id_ae7cb51e,0.000454,-0.034732
2,id_e59f7730,0.000454,-0.034732
3,id_b9011c86,0.046222,0.043726
4,id_caaeb9f8,0.045682,0.039802


In [ ]:
m2_lgbm_test.to_csv('final_sub_lgbm.csv', index=False)